__`randomizer.ipynb`__

This notebook is used to generate two randomized lists:  
1. A list of stop words to feed into a query
2. A list of blocks of time

The purpose of these lists are for randomized sampling of tweets from the Twitter API's full archive search.

One requirement of the Twitter API's full archive search endpoint is the use of at least one "standalone" operator. A simple example of a standalone operator is a user's Twitter handle. We're interested in finding a wide array of randomized tweets made by verified accounts, so we're utilizing the __keyword__ standalone operator to search for tweets whose text contains at least one of a list of keywords. To minimize bias in selecting keywords, however, the keywords we're using will be a long list of English language stop words (e.g. "the", "a", "why").

Our corpus of troll tweets span the time period of 2013-2017. If we were to use this query across that entire time span, it would potentially match *hundreds of millions of tweets*. Rather than trying an acquire-first-then-sampel approach, we're applying an approach to sample-first-then-acquire. 

The methodology to "sample first" is to break up the four year span of 2013 to 2017 into four-hour-long blocks of time (10,956 time blocks total) and then randomly sample (without replacement) time blocks from there. The order of time blocks acquired will also be random such that we do not need to set a specific number of time blocks in advance. Instead we will continue aquiring tweets (with above query) within sampled time blocks until we have acquired a sufficient number of tweets.

So in summary:  
- Query the full archive of tweets over the time period of 2013-2017
- Search for tweets containing any of a large list of randomly chosen English stop words
- Sample from the larger four-year span by identifying multiple randomly chosen, randomly ordered, four-hour-long time blocks
- Submit the query for as many four-hour-long time blocks as necessary to acquire a sufficient number of tweets

# Stop Words List

Using the `stopwords` corpus from the NLTK package allows us to quickly obtain a list of common stop words. NLTK is a prominent and actively developed package for use in NLP applications.

The first time this notebook is run on a particular environment/interpreter, the `stopwords` corpus needs to be downloaded. Uncomment the third line in the next cell to do this.

In [1]:
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')     # only needs to be run once

import random

In [2]:
stopwords_english = stopwords.words('english')

Randomly sample (without replacement) 100 of the (currently) 179 stop words provided by NLTK.

For use in the Twitter API, a quick sanitization is needed to wrap the strings `and` and `or` in double quotes. This avoids them being intepretted as logical operators. The Twitter API throws an error if this step is not performed.

Twitter API queries also have a maximum length of 1,024 characters, so a quick check of the query length is performed.

In [3]:
sw_random_subset = random.sample(stopwords_english, k=100)

sw_subset_sanitzed = []

for word in sw_random_subset:
    if word.lower() in ["and", "or"]:
        sw_subset_sanitzed.append(f"\"{word.lower()}\"")
    else:
        sw_subset_sanitzed.append(word.lower())

query_text = f"({' OR '.join(sw_subset_sanitzed)})"

len(query_text)

821

To finalize the query, we append three "conjunction-required" operators to our list of stop words from prior cell.

- `-is:nullcast` filters out promoted/ad tweets (the leading "`-`" negates in this context)
- `is:verified` filters for tweets made by verified accounts
- `lang:en` filters for tweets identified by Twitter as being in the English language

In [4]:
query_text += " -is:nullcast is:verified lang:en"

print("characters:", len(query_text))
print(query_text)

characters: 854
(why OR than OR y OR as OR hasn't OR you'd OR ma OR doesn't OR don't OR weren't OR couldn OR shan't OR ve OR himself OR have OR her OR out OR very OR me OR because OR wouldn OR am OR isn't OR such OR but OR should've OR didn OR hadn OR not OR an OR now OR during OR aren't OR "and" OR she OR we OR re OR only OR having OR yourselves OR d OR further OR when OR needn't OR theirs OR they OR no OR hers OR below OR being OR here OR then OR it OR mightn OR by OR to OR between OR their OR ain OR t OR just OR "or" OR ourselves OR o OR shouldn't OR itself OR on OR at OR wasn't OR didn't OR those OR its OR haven OR are OR themselves OR few OR down OR hadn't OR my OR most OR who OR if OR yours OR been OR has OR into OR any OR were OR you OR is OR from OR the OR don OR needn OR it's OR i OR shouldn OR with OR does OR until) -is:nullcast is:verified lang:en


# Time Block List

Use Python's built-in `datetime` package to generate the full list of non-overlapping, four-hour-long blocks of time from the time span starting 01-Jan-2013 (inclusive) and ending 01-Jan-2018 (exclusive).

The time blocks are represented in the list by their starting date/time.

In [5]:
from datetime import datetime, timedelta

# make list of blocks of time
start_date: datetime = datetime(year=2013, month=1, day=1, hour=0, minute=0, second=0)
end_date: datetime = datetime(year=2018, month=1, day=1, hour=0, minute=0, second=0)

four_hours: timedelta = timedelta(hours=4)

timeblock_list: list[datetime] = [start_date]
current_block_end: datetime = start_date + four_hours

while (current_block_end < end_date):
    timeblock_list.append(current_block_end)
    current_block_end += four_hours

print(timeblock_list[-2:])

[datetime.datetime(2017, 12, 31, 16, 0), datetime.datetime(2017, 12, 31, 20, 0)]


From this complete list of time blocks, sample a subset of time blocks.

Then print a formatted list of these time blocks.

In [6]:
timeblock_random_subset: list[datetime] = random.sample(timeblock_list, k=20)

for timeblock in timeblock_random_subset:
    end_time: datetime = timeblock + four_hours
    print(timeblock.strftime(r"%Y-%m-%d %I:%M %p"), "-- to --", end_time.strftime(r"%Y-%m-%d %I:%M %p"))

2014-04-23 08:00 AM -- to -- 2014-04-23 12:00 PM
2017-12-03 12:00 AM -- to -- 2017-12-03 04:00 AM
2015-04-26 08:00 PM -- to -- 2015-04-27 12:00 AM
2017-03-20 12:00 AM -- to -- 2017-03-20 04:00 AM
2013-12-13 08:00 AM -- to -- 2013-12-13 12:00 PM
2017-06-29 08:00 PM -- to -- 2017-06-30 12:00 AM
2017-04-14 12:00 PM -- to -- 2017-04-14 04:00 PM
2016-01-19 08:00 AM -- to -- 2016-01-19 12:00 PM
2013-07-06 12:00 AM -- to -- 2013-07-06 04:00 AM
2015-08-18 04:00 AM -- to -- 2015-08-18 08:00 AM
2013-07-03 08:00 PM -- to -- 2013-07-04 12:00 AM
2016-12-01 08:00 AM -- to -- 2016-12-01 12:00 PM
2017-06-26 12:00 PM -- to -- 2017-06-26 04:00 PM
2015-05-04 04:00 PM -- to -- 2015-05-04 08:00 PM
2017-12-16 04:00 PM -- to -- 2017-12-16 08:00 PM
2016-05-11 12:00 AM -- to -- 2016-05-11 04:00 AM
2016-11-30 12:00 PM -- to -- 2016-11-30 04:00 PM
2013-09-20 08:00 AM -- to -- 2013-09-20 12:00 PM
2016-07-03 08:00 PM -- to -- 2016-07-04 12:00 AM
2014-05-09 04:00 PM -- to -- 2014-05-09 08:00 PM


We now have the query text and a list of time blocks to use as we send requests to the Twitter API.